In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#foldername= "/content/drive/My Drive/nlpproject/bertsequenceclassification/"
foldername= "/content/drive/My Drive/Project/extractedfeaturewithbert/"
#foldername= "/content/drive/My Drive/extractedfeaturewithbert/"


In [ ]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

#import keras
#from keras.models import Model
#import keras.backend as K
#from keras.callbacks import ModelCheckpoint
#from keras.models import load_model


In [ ]:
class CFG():
  max_len = 50
  #model = "bert-base-uncased"
  model = "distilbert-base-uncased"
  lr=1e-4
  min_lr=1e-10
  batch_size=16
  epoch=1000
  embedding_dim=512
  hidden_dim=256
  drop=0.15
cfg=CFG()

In [ ]:
class Ds(Dataset):
    def __init__(self, path, tokenizer, i, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=i #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        t=item["type"]
        labels=self.str2label(t)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1
        else:
            return 0
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]



In [ ]:
#from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
#tokenizer = BertTokenizer.from_pretrained(cfg.model)
tokenizer = DistilBertTokenizer.from_pretrained(cfg.model)
config = AutoConfig.from_pretrained(cfg.model)
#bertmodel= BertForSequenceClassification.from_pretrained(cfg.model, config=config)
distilbertmodel= DistilBertForSequenceClassification.from_pretrained(cfg.model, config=config)
#bertmodel = AutoModelForSequenceClassification.from_pretrained(model_path)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

In [ ]:
for name, param in distilbertmodel.named_parameters():
  print(name)
  if name.startswith('distilbert.transformer.layer.5'):
    param.requires_grad = True
  else:
    if name.startswith('distilbert.transformer.layer.')
      param.requires_grad = False
    else:
      param.requires_grad = True
  

distilbert.embeddings.word_embeddings.weight
distilbert.embeddings.position_embeddings.weight
distilbert.embeddings.LayerNorm.weight
distilbert.embeddings.LayerNorm.bias
distilbert.transformer.layer.0.attention.q_lin.weight
distilbert.transformer.layer.0.attention.q_lin.bias
distilbert.transformer.layer.0.attention.k_lin.weight
distilbert.transformer.layer.0.attention.k_lin.bias
distilbert.transformer.layer.0.attention.v_lin.weight
distilbert.transformer.layer.0.attention.v_lin.bias
distilbert.transformer.layer.0.attention.out_lin.weight
distilbert.transformer.layer.0.attention.out_lin.bias
distilbert.transformer.layer.0.sa_layer_norm.weight
distilbert.transformer.layer.0.sa_layer_norm.bias
distilbert.transformer.layer.0.ffn.lin1.weight
distilbert.transformer.layer.0.ffn.lin1.bias
distilbert.transformer.layer.0.ffn.lin2.weight
distilbert.transformer.layer.0.ffn.lin2.bias
distilbert.transformer.layer.0.output_layer_norm.weight
distilbert.transformer.layer.0.output_layer_norm.bias
distil

In [ ]:
#path=foldername+"dataset2.csv"
path=foldername+"balanced_EI_dataset1.csv"
dataset_EI=Ds(path, tokenizer, i=0)

#dataset_SN=Ds_SN(path, tokenizer)
#dl_SN=getdl(dataset_SN)

#dataset_TF=Ds_TF(path, tokenizer)
#dl_TF=getdl(dataset_TF)

#dataset_JP=Ds_JP(path, tokenizer)
#dl_JP=getdl(dataset_JP)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
device

'cuda'

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.2726])).to(device)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1)).to(device)
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=foldername+"test_trainer", 
                                  evaluation_strategy="epoch", 
                                  num_train_epochs=30, 
                                  per_device_train_batch_size=32, 
                                  per_device_eval_batch_size=32,
                                  save_steps=1e5,
                                  weight_decay=0.01,
                                  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
total_len=len(dataset_EI)
train_len=int(len(dataset_EI)*0.9)
val_len=int(total_len-train_len)
[train_ds, val_ds]=torch.utils.data.random_split(dataset_EI, [train_len, val_len])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
trainer = CustomTrainer(
    #model=bertmodel,
    model=distilbertmodel,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 166779
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 156360
  Number of trainable parameters = 592130


Epoch,Training Loss,Validation Loss,Accuracy
1,0.544500,0.542906,0.498624
2,0.539900,0.542737,0.498624
3,0.541200,0.542591,0.498786
4,0.537100,0.544046,0.501106
5,0.538500,0.541049,0.498678
6,0.542000,0.542226,0.499811
7,0.543500,0.541871,0.498948
8,0.538700,0.540731,0.499110
9,0.535900,0.540689,0.500027
10,0.530300,0.540154,0.500189


***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch si

Epoch,Training Loss,Validation Loss,Accuracy
1,0.544500,0.542906,0.498624
2,0.539900,0.542737,0.498624
3,0.541200,0.542591,0.498786
4,0.537100,0.544046,0.501106
5,0.538500,0.541049,0.498678
6,0.542000,0.542226,0.499811
7,0.543500,0.541871,0.498948
8,0.538700,0.540731,0.499110
9,0.535900,0.540689,0.500027
10,0.530300,0.540154,0.500189


***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
Saving model checkpoint to /content/drive/My Drive/Project/extractedfeaturewithbert/test_trainer/checkpoint-100000
Configuration saved in /content/drive/My Drive/Project/extractedfeaturewithbert/test_trainer/checkpoint-100000/config.json
Model weights saved in /content/drive/My Drive/Project/extractedfeaturewithbert/test_trainer/checkpoint-100000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  Batch size = 32
***** Running Evaluation *****
  Num examples = 18531
  B

TrainOutput(global_step=156360, training_loss=0.535703285492482, metrics={'train_runtime': 11664.1435, 'train_samples_per_second': 428.953, 'train_steps_per_second': 13.405, 'total_flos': 6.4724942227878e+16, 'train_loss': 0.535703285492482, 'epoch': 30.0})